<h1 style="font-size:400%; text-align:center">Functions description</h1>

In [7]:
import os
import sys

from IPython.display import HTML

display(HTML('''
<style>
    h1 {text-align:center}
</style>
'''))

souce_code_filepath = os.path.join(
    os.getcwd(), "NBRB_bki_xml_read"
)
if not souce_code_filepath in sys.path:
    sys.path.append(souce_code_filepath)

%load_ext autoreload
%autoreload 2
    
from NBRB_bki_xml_read.common_tools import dict_from_xmlfile,\
                                           recursive_dict_reader,\
                                           dict_reading_decorator,\
                                           sub_dict_reader
from NBRB_bki_xml_read.individual_read import read_client,\
                                              read_result

%aimport NBRB_bki_xml_read.individual_read
%aimport NBRB_bki_xml_read.common_tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# `common_tools`

### `dict_from_xmlfile`

Сonsider an xml file:

In [2]:
with open("example_files/some_xml_file.xml", "rb") as file:
    print(file.read().decode('windows-1251'))

<?xml version="1.0" encoding="windows-1251"?>
<root xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <tbsData>

    <doc_purpose>KBO11012</doc_purpose>

    <message system="AISKB">
      <MessageReference>111111111111111111111111111111111111111111</MessageReference>
      <date>11.02.2023</date>
      <source>
        <bic>1111111111111</bic>
        <bankname>Some bankname</bankname>
      </source>
      <Response type="11015" name="getfullhistscorefiz">
        <MessageLinkReference>C111111111111111111111111111111</MessageLinkReference>
        <result>
          <completecode class="A">
            <errorcode>0</errorcode>
            <description>Запрос успешно обработан</description>
          </completecode>
          <client>
            <titul>
              <FIO>
                <lastname>lastname</lastname>
                <firstname>firstname</firstname>
                <middlename>middlename</middlename>
              </FIO>
              <nationality>
           

As dict:

In [3]:
dict_from_xmlfile("example_files/some_xml_file.xml")

{'root': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  'tbsData': {'doc_purpose': 'KBO11012',
   'message': {'@system': 'AISKB',
    'MessageReference': '111111111111111111111111111111111111111111',
    'date': '11.02.2023',
    'source': {'bic': '1111111111111', 'bankname': 'Some bankname'},
    'Response': {'@type': '11015',
     '@name': 'getfullhistscorefiz',
     'MessageLinkReference': 'C111111111111111111111111111111',
     'result': {'completecode': {'@class': 'A',
       'errorcode': '0',
       'description': 'Запрос успешно обработан'},
      'client': {'titul': {'FIO': {'lastname': 'lastname',
         'firstname': 'firstname',
         'middlename': 'middlename'},
        'nationality': {'codCountry': '112',
         'nameCountry': 'Республика Беларусь'},
        'sex': 'M',
        'IDNumber': '11111111111111111111',
        'birthday': '01.01.2000'},
       'registrationplace': {'country': {'codCountry': '112',
         'nameCountry': 'Республика Беларусь

### `recursive_dict_reader`

In [4]:
example_dictionary = {
    "level1_value" : "some_value",
    "level1_dict" : {
        "level2_dict" : {
            "level3_value" : "some_value"
        },
        "level2_value" : "some_value",
        "level2_value2" : "some_value2"
    }
}

recursive_dict_reader(example_dictionary, keys_separator=".")

{'level1_value': 'some_value',
 'level1_dict.level2_dict.level3_value': 'some_value',
 'level1_dict.level2_value': 'some_value',
 'level1_dict.level2_value2': 'some_value2'}

### `dict_reading_decorator`

In [5]:
@dict_reading_decorator
def some_read_dict_func(d):
    return d["my_key"]

print("Сorrect dictionary")
print(some_read_dict_func({"my_key":5}))
print("Error dictionary")
print(some_read_dict_func({"other_key":5}))

Сorrect dictionary
5
Error dictionary
{}


### `sub_dict_reader`

We will use the following dictionary as an example:

In [6]:
example_dictionary = {
    "level1_value" : "some_value",
    "level1_dict" : {
        "level2_dict" : {
            "level3_value" : "some_value"
        },
        "level2_value" : "some_value",
        "level2_value2" : "some_value2"
    }
}

Suppose you want to extract `level1_dict` as a flat dictionary, you can use `recursive_dict_reader` to do this:

In [7]:
recursive_dict_reader(example_dictionary["level1_dict"])

{'level2_dict/level3_value': 'some_value',
 'level2_value': 'some_value',
 'level2_value2': 'some_value2'}

But it's really common when you want to store `level1_dict` names in flat indexes. To achieve this you will write something like this:

In [8]:
{
    "level1_dict/" + key : value
    for key,value in 
    recursive_dict_reader(example_dictionary["level1_dict"]).items()
}

{'level1_dict/level2_dict/level3_value': 'some_value',
 'level1_dict/level2_value': 'some_value',
 'level1_dict/level2_value2': 'some_value2'}

You can achieve the same result by using `sub_dict_reader`:

In [9]:
sub_dict_reader(example_dictionary, "level1_dict", keys_separator = "|")

{'level1_dict|level2_dict|level3_value': 'some_value',
 'level1_dict|level2_value': 'some_value',
 'level1_dict|level2_value2': 'some_value2'}

# `individual_read`

### `read_client`

In [13]:
client_example = {
    'titul': {
        'FIO': {
            'lastname': 'lastname',
            'firstname': 'firstname',
            'middlename': 'middlename'
        },
        'nationality': {
            'codCountry': '112', 
            'nameCountry': 'Республика Беларусь'
        },
        'sex': 'M',
        'IDNumber': '1111111111111',
        'birthday': '01.01.2000'
    },
    'registrationplace': {
        'country': {
            'codCountry': '112',
            'nameCountry': 'Республика Беларусь'},
            'nationalplace': {
                'codSettlement': '1212000000',
                'nameSettlement': 'обл. Брестская, р-н Брестский район'
            }
    },
    'scoring': {
        'ratingclass': 'B1', 
        'finalscore': '325', 
        'finalPPD': '0.50'
    },
}

read_client(client_example, keys_separator = "*")

{'client*titul*FIO*lastname': 'lastname',
 'client*titul*FIO*firstname': 'firstname',
 'client*titul*FIO*middlename': 'middlename',
 'client*titul*nationality*codCountry': '112',
 'client*titul*nationality*nameCountry': 'Республика Беларусь',
 'client*titul*sex': 'M',
 'client*titul*IDNumber': '1111111111111',
 'client*titul*birthday': '01.01.2000',
 'client*registrationplace*country*codCountry': '112',
 'client*registrationplace*country*nameCountry': 'Республика Беларусь',
 'client*registrationplace*nationalplace*codSettlement': '1212000000',
 'client*registrationplace*nationalplace*nameSettlement': 'обл. Брестская, р-н Брестский район',
 'client*scoring*ratingclass': 'B1',
 'client*scoring*finalscore': '325',
 'client*scoring*finalPPD': '0.50'}

### `read_result`

In [10]:
result = {
    'completecode': {
        '@class': 'A',
        'errorcode': '0',
        'description': 'Запрос успешно обработан'
    },
    'client': {
        'titul': {
            'FIO': {
                'lastname': 'lastname',
                'firstname': 'firstname',
                'middlename': 'middlename'
            },
            'nationality': {
                'codCountry': '112', 
                'nameCountry': 'Республика Беларусь'
            },
           'sex': 'M',
           'IDNumber': '**************',
           'birthday': '01.01.2000'
        },
        'registrationplace': {
            'country': {
                'codCountry': '112',
                'nameCountry': 'Республика Беларусь'
            },
            'nationalplace': {
                'codSettlement': '5000000000',
                'nameSettlement': 'г. Минск'
            }
        },
        'scoring': {
            'ratingclass': 'B2', 
            'finalscore': '288', 
            'finalPPD': '0.95'
        }
    }
}

read_result(result, keys_separator = "_")

{'result_completecode_@class': 'A',
 'result_completecode_errorcode': '0',
 'result_completecode_description': 'Запрос успешно обработан',
 'result_client_titul_FIO_lastname': 'lastname',
 'result_client_titul_FIO_firstname': 'firstname',
 'result_client_titul_FIO_middlename': 'middlename',
 'result_client_titul_nationality_codCountry': '112',
 'result_client_titul_nationality_nameCountry': 'Республика Беларусь',
 'result_client_titul_sex': 'M',
 'result_client_titul_IDNumber': '**************',
 'result_client_titul_birthday': '01.01.2000',
 'result_client_registrationplace_country_codCountry': '112',
 'result_client_registrationplace_country_nameCountry': 'Республика Беларусь',
 'result_client_registrationplace_nationalplace_codSettlement': '5000000000',
 'result_client_registrationplace_nationalplace_nameSettlement': 'г. Минск',
 'result_client_scoring_ratingclass': 'B2',
 'result_client_scoring_finalscore': '288',
 'result_client_scoring_finalPPD': '0.95'}